# Data analysis

<div class="alert alert-danger">
<b>WARNING!</b> <p>This notebook was thought for the first groundtruth proposal (predicting the given sensor scores).</p> However, <b>this approach has been discarded</b>, and instead, work is being done on the second proposal (predicting X-ray density). Please check the analysis notebook in <tt>second_proposed_gt</tt>
</div>

In [1]:
import pandas as pd
import os

Now that the analysis of the different pole images is reaching an advanced stage, it is a good moment to begin carrying out a proper data analysis process on the features extracted so far.

## Feature merging

The first step for analysing the features is merging everything together on a single CSV file

In [8]:
# Read full DF if it is present, merge all other DFs instead
if ('full_df.csv' not in os.listdir()):
    df_list = []

    df_paths = [path for path in os.listdir() if path.lower().endswith('.csv') and path != "full_df.csv"]
    for path in df_paths:
        df = pd.read_csv(path, index_col='id')
        df = df.drop('Unnamed: 0', axis=1)
        df_list.append(df)

    full_df = pd.concat(df_list, axis=1, join='inner')
    full_df.to_csv('full_df.csv')

else:
    full_df = pd.read_csv('full_df.csv', index_col='id')

## Dataset analysis proper

Now that all the information is in one DataFrame, it is now possible to carry out the data analysis process on it:

In [11]:
full_df

,kernel_0_mean_0_degrees,kernel_0_std_0_degrees,kernel_0_mode_0_degrees,kernel_0_mean_90_degrees,kernel_0_std_90_degrees,kernel_0_mode_90_degrees,kernel_0_mean_180_degrees,kernel_0_std_180_degrees,kernel_0_mode_180_degrees,kernel_0_mean_270_degrees,...,g_hist_mean_180_degrees,g_hist_std_180_degrees,b_hist_mean_180_degrees,b_hist_std_180_degrees,r_hist_mean_270_degrees,r_hist_std_270_degrees,g_hist_mean_270_degrees,g_hist_std_270_degrees,b_hist_mean_270_degrees,b_hist_std_270_degrees
id,,,,,,,,,,,,,,,,,,,,,
0,0.254073,0.169437,0.175,0.252666,0.160712,0.145,0.311594,0.191645,0.225,0.251231,...,22.354347,73.765120,33.327347,89.48301,23.998850,74.826256,23.837667,74.04890,29.674149,79.30145
30,0.275938,0.170302,0.085,0.382975,0.205917,0.135,0.410874,0.233415,0.105,0.345359,...,119.497970,141.465840,128.071090,133.19722,85.812515,138.904340,96.649120,141.10794,103.375275,135.96727
41,0.241549,0.164686,0.125,0.285987,0.182777,0.125,0.279583,0.175961,0.115,0.267159,...,74.448350,125.971664,95.981110,132.78305,36.249535,89.356760,45.171640,98.90569,68.635770,113.76341
6,0.309093,0.183176,0.105,0.233579,0.133327,0.085,0.347708,0.201557,0.195,0.312908,...,85.711494,129.029560,109.869570,142.78038,77.993510,120.583980,93.702675,130.32787,101.291970,129.39589
5,0.308027,0.173186,0.215,0.298685,0.186155,0.125,0.305078,0.165581,0.145,0.305544,...,91.452420,135.622010,95.183945,131.20311,66.083310,116.386734,83.667206,126.72504,101.217910,132.11688


As it can be seen, this dataset only has 5 observations, corresponding to the 5 poles, and 623 columns, corresponding to statistical values coming from all the applied techniques: 

- Means, standard deviations and modes from the Gabor kernel activations
- Means and standard deviations from the color and gray histograms
- Spectra of the hyperspectral scans
- Information about the scores and statuses of the poles

With such high dimensionality, it is near impossible to train good algorithms to predict the scores and statuses of the poles. It is therefore desirable to remove correlated variables prior to continuing the analysis.

Before doing this